# AutoTokenizer.from_pretrained(MODEL_NAME)

In [5]:
from transformers import AutoTokenizer
import pickle as pickle
import os
import pandas as pd
import torch

def load_data(dataset_dir):
    """ csv 파일을 경로에 맡게 불러 옵니다. """
    pd_dataset = pd.read_csv(dataset_dir)
    dataset = preprocessing_dataset(pd_dataset)
  
    return dataset

def tokenized_dataset(dataset, tokenizer):
    """ tokenizer에 따라 sentence를 tokenizing 합니다."""
    concat_entity = []
    for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
        temp = ''
        temp = e01 + '[SEP]' + e02
        concat_entity.append(temp)
    tokenized_sentences = tokenizer(
        concat_entity,
        list(dataset['sentence']),
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256,
        add_special_tokens=True,
    )
    return tokenized_sentences

In [8]:
def label_to_num(label):
    num_label = []
    with open('dict_label_to_num.pkl', 'rb') as f:
        dict_label_to_num = pickle.load(f)
    for v in label:
        num_label.append(dict_label_to_num[v])

    return num_label

def preprocessing_dataset(dataset):
    """ 처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
    subject_entity = []
    object_entity = []
    for i,j in zip(dataset['subject_entity'], dataset['object_entity']):
        i = i[1:-1].split(',')[0].split(':')[1]
        j = j[1:-1].split(',')[0].split(':')[1]

        subject_entity.append(i)
        object_entity.append(j)
    out_dataset = pd.DataFrame({'id':dataset['id'], 'sentence':dataset['sentence'],'subject_entity':subject_entity,'object_entity':object_entity,'label':dataset['label'],})
    return out_dataset

MODEL_NAME = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print("loading data")
train_dataset = load_data("train.csv")

print("label to num")
train_label = label_to_num(train_dataset['label'].values)

print("tokenized dataset")
tokenized_train = tokenized_dataset(train_dataset, tokenizer)


print(type(tokenized_train))
print(list(tokenized_train))
# print(tokenized_train)
# print(tokenized_train['input_ids'][1])

print(train_dataset['sentence'])

loading data
label to num
tokenized dataset
<class 'transformers.tokenization_utils_base.BatchEncoding'>
['input_ids', 'token_type_ids', 'attention_mask']
0        〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...
1        호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...
2        K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...
3        균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...
4        1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...
                               ...                        
32465    한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...
32466    법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...
32467    완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...
32468    중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...
32469    화순군(군수 구충곤)은 17일 동면의 이장 20여 명이 코로나 19 예방을 위해 버...
Name: sentence, Length: 32470, dtype: object


TypeError: tokenized_dataset() missing 1 required positional argument: 'tokenizer'

In [11]:
train_dataset.iloc[1]

id                                                                1
sentence          호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...
subject_entity                                              '민주평화당'
object_entity                                                '대안신당'
label                                                   no_relation
Name: 1, dtype: object

In [16]:
pd.DataFrame(train_dataset.iloc[1]).T

,id,sentence,subject_entity,object_entity,label
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,'민주평화당','대안신당',no_relation


In [14]:
print(type(train_dataset.iloc[1]))
print(type(train_dataset))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [17]:
tokenized_dataset(pd.DataFrame(train_dataset.iloc[1]).T, tokenizer)

{'input_ids': tensor([[    2,    11,  3772,  2139,  2267,  2481,    11,     3,    11,  5605,
          2250,  2481,    11,     3,  6409,  2052,  4568,  2179,  6417,  2044,
          2315,  2481,   100,  5605,  2250,  2481,   100,  3772,  2139,  2267,
          2481,  2052, 16489,   711,  2170, 12827,  2097,  8646,  2481,    12,
         15283,    13,  3603,  1528,  2554,  2065,  4538,    18,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}

In [22]:
tokenizer.tokenize("바른미래당", add_special_tokens = True)

['[CLS]', '바른', '##미', '##래', '##당', '[SEP]']

In [23]:
tokenizer.encode("바른미래당")

[2, 6417, 2044, 2315, 2481, 3]